In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install wget

In [ ]:
import os
import pandas as pd
import random
import zipfile

In [ ]:
!python -m wget https://filr.hs-offenburg.de/filr/public-link/file-download/0dcf8b85882ae199018870bceddf437c/5426/-2254907074900866378/test_data.zip

In [ ]:
!python -m wget https://zenodo.org/record/7869954/files/products_leaflets_256.zip?download=1

In [ ]:
path_to_zip_file = 'products_leaflets_256.zip'
directory_to_extract_to = '.'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [ ]:
path_to_zip_file = 'test_data.zip'
directory_to_extract_to = '.'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [ ]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.utils import image_dataset_from_directory



# Verzeichnis mit den Trainingsdaten
train_directory = 'products_leaflets_256/train'

# Bildgröße und Anzahl der Klassen
image_size = (256, 256)
num_classes = len(os.listdir(train_directory))

# Liste für Bilder und Labels
images = []
labels = []
train_dataset = image_dataset_from_directory(
    train_directory,
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)





In [ ]:
import tensorflow.keras
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.preprocessing import image


#Densenet121 with defaul parameters
model = DenseNet121(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=(256, 256, 3),
    pooling=None,
    classes=832,
    classifier_activation="softmax",
)
model2 = model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')


# Trainieren des Modells mit 10 durchläufen
model.fit(train_dataset, batch_size=32, epochs=50)

In [ ]:
# Liste für Bildnamen und Vorhersagen
image_names = []
label_predictions = []
test_directory = 'test_data'

model.save_weights('my_model_weights.h5')

# Durchlaufe das Testverzeichnis und lade die Bilder
for root, dirs, files in os.walk(test_directory):
    for file in files:
        if file.endswith('.jpg'):
            image_path = os.path.join(root, file)
            image = Image.open(image_path).convert('RGB')
            image = image.resize(image_size)
            image_array = np.array(image)
            image_names.append(file)
            label_prediction = model.predict(np.expand_dims(image_array, axis=0))
            label_predictions.append(np.argmax(label_prediction))

# Erstelle den DataFrame und speichere die Vorhersagen in einer CSV-Datei
df = pd.DataFrame(columns=['image_id', 'label'])
df['image_id'] = image_names
df['label'] = label_predictions
df.to_csv('submission.csv', index=False)

# Neuer Abschnitt